<a href="https://colab.research.google.com/github/akuma527/IProjects/blob/master/I%26D_Whale/Training_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%cd /content/drive/My Drive/Colab Notebooks/I&D_Whale/data/

/content/drive/My Drive/Colab Notebooks/I&D_Whale/data


In [0]:
import pandas as pd
import os
import cv2
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Importing Data

In [0]:
df = pd.read_csv('data_prep_fin.csv')

In [10]:
df.head()

,Image,Similar_Images,Id,fil_path
0,IMG_2270-16.jpg,[''],1756,train/1756/IMG_2270-16.jpg
1,PM-WWA-20050413-003a.jpg,[''],2824,train/2824/PM-WWA-20050413-003a.jpg
2,PM-WWA-20050413-003b.jpg,['PM-WWA-20050413-029.jpg'],2764,train/2764/PM-WWA-20050413-003b.jpg
3,PM-WWA-20050413-029.jpg,['PM-WWA-20050413-003b.jpg'],2764,train/2764/PM-WWA-20050413-029.jpg
4,PM-WWA-20050413-006.jpg,['PM-WWA-20050413-007.jpg'],2825,train/2825/PM-WWA-20050413-006.jpg


In [0]:
df['Similar_Images'] = df['Similar_Images'].replace(str(['']), np.nan)

# Creating Image and Similar Image data set

In [51]:
df.head()

,Image,Similar_Images,Id,fil_path
0,IMG_2270-16.jpg,NaN,1756,train/1756/IMG_2270-16.jpg
1,PM-WWA-20050413-003a.jpg,NaN,2824,train/2824/PM-WWA-20050413-003a.jpg
2,PM-WWA-20050413-003b.jpg,['PM-WWA-20050413-029.jpg'],2764,train/2764/PM-WWA-20050413-003b.jpg
3,PM-WWA-20050413-029.jpg,['PM-WWA-20050413-003b.jpg'],2764,train/2764/PM-WWA-20050413-029.jpg
4,PM-WWA-20050413-006.jpg,['PM-WWA-20050413-007.jpg'],2825,train/2825/PM-WWA-20050413-006.jpg


In [0]:
coupled = df[~df['Similar_Images'].isna()].reset_index(drop=True)

In [55]:
coupled.head()

,Image,Similar_Images,Id,fil_path
0,PM-WWA-20050413-003b.jpg,['PM-WWA-20050413-029.jpg'],2764,train/2764/PM-WWA-20050413-003b.jpg
1,PM-WWA-20050413-029.jpg,['PM-WWA-20050413-003b.jpg'],2764,train/2764/PM-WWA-20050413-029.jpg
2,PM-WWA-20050413-006.jpg,['PM-WWA-20050413-007.jpg'],2825,train/2825/PM-WWA-20050413-006.jpg
3,PM-WWA-20050413-007.jpg,['PM-WWA-20050413-006.jpg'],2825,train/2825/PM-WWA-20050413-007.jpg
4,PM-WWA-20050413-009.jpg,['PM-BIO-20050413-049.jpg'],2826,train/2826/PM-WWA-20050413-009.jpg


In [71]:
coupled.shape

(2302, 4)

In [0]:
coupled['Similar_Images'] = coupled['Similar_Images'].apply(lambda x: eval(x))

In [0]:
def _de_norm(df):
    lst_col = 'Similar_Images'
    r = pd.DataFrame({col: np.repeat(df[col].values, df[lst_col].str.len())
                      for col in df.columns.drop(lst_col)}
                     ).assign(**{lst_col: np.concatenate(df[lst_col].values)})[df.columns]
    return r

In [0]:
de_coupled = _de_norm(coupled)

In [107]:
de_coupled['Id'].nunique()

577

In [0]:
no_coupled = df[df['Similar_Images'].isna()].reset_index(drop=True)

In [109]:
no_coupled.shape

(1660, 4)

In [110]:
no_coupled.head()

,Image,Similar_Images,Id,fil_path
0,IMG_2270-16.jpg,NaN,1756,train/1756/IMG_2270-16.jpg
1,PM-WWA-20050413-003a.jpg,NaN,2824,train/2824/PM-WWA-20050413-003a.jpg
2,PM-WWA-20050413-012.jpg,NaN,2827,train/2827/PM-WWA-20050413-012.jpg
3,PM-BIO-20050414-116.jpg,NaN,2830,train/2830/PM-BIO-20050414-116.jpg
4,PM-WWA-20050414-096.jpg,NaN,2829,train/2829/PM-WWA-20050414-096.jpg


In [0]:
shifted = no_coupled.shift(-1).drop(columns=['Similar_Images'])

In [0]:
shifted.columns=['Image_2', 'Id_2', 'fil_path_2']

In [0]:
shifted.iloc[-1] = no_coupled[['Image', 'Id', 'fil_path']].iloc[0].tolist()

In [0]:
pd.merge(no_coupled[['Image', 'Id', 'fil_path']], shifted, left_index=True, right_index=True).to_csv('not_similar.csv', index=False)

In [150]:
de_coupled.head()

,Image,Similar_Images,Id,fil_path
0,PM-WWA-20050413-003b.jpg,PM-WWA-20050413-029.jpg,2764,train/2764/PM-WWA-20050413-003b.jpg
1,PM-WWA-20050413-029.jpg,PM-WWA-20050413-003b.jpg,2764,train/2764/PM-WWA-20050413-029.jpg
2,PM-WWA-20050413-006.jpg,PM-WWA-20050413-007.jpg,2825,train/2825/PM-WWA-20050413-006.jpg
3,PM-WWA-20050413-007.jpg,PM-WWA-20050413-006.jpg,2825,train/2825/PM-WWA-20050413-007.jpg
4,PM-WWA-20050413-009.jpg,PM-BIO-20050413-049.jpg,2826,train/2826/PM-WWA-20050413-009.jpg


In [167]:
train = pd.read_csv('train.csv')
train.head()

,Image,Id,fil_path
0,PM-WWA-20050416-037.jpg,-1,train/-1/PM-WWA-20050416-037.jpg
1,PM-WWA-20050513-229.jpg,-1,train/-1/PM-WWA-20050513-229.jpg
2,PM-WWA-20050513-243.jpg,-1,train/-1/PM-WWA-20050513-243.jpg
3,PM-WWA-20050520-134.jpg,-1,train/-1/PM-WWA-20050520-134.jpg
4,PM-WWA-20050612-047.jpg,-1,train/-1/PM-WWA-20050612-047.jpg


In [0]:
fin_train = pd.merge(de_coupled, train, left_on='Similar_Images', right_on='Image', how='left')

In [0]:
fin_train = fin_train[['Image_x', 'Id_x', 'fil_path_x', 'Image_y', 'Id_y',
       'fil_path_y']]

In [0]:
fin_train.columns = ['Image', 'Id', 'fil_path', 'Image_2', 'Id_2', 'fil_path_2']

In [0]:
fin_train.to_csv('final_train.csv', index=False)